In [122]:
import pandas as pd
import ast
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [123]:
def clean_array_string(array_string):
    # Replace newlines and multiple spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', array_string)
    # Add a comma after each number except the last one
    cleaned_string_with_commas = re.sub(r'([-\d.e]+)(?=\s)', r'\1,', cleaned_string)
    return cleaned_string_with_commas


In [124]:

politician_selection = 'Sen. David Perdue'


In [125]:
transactions = pd.read_csv('final_data/transactions_final.csv')
committee_assignments = pd.read_csv('final_data/committee_assignments_final.csv')
subcommittee_assignments = pd.read_csv('final_data/subcommittee_assignments_final.csv')
transactions['transaction_date'] = pd.to_datetime(transactions['combined_transaction_date'])
transactions[transactions['display_name'] == politician_selection]

,Unnamed: 0,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,...,ptr_link,amount_formatted,member_id,uuid,congress,industry_description,sector_description,string_object,embedding,transaction_date
7589,7589,Sen. David Perdue,david perdue,2020-11-04,11/04/2020,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,...,https://efdsearch.senate.gov/search/view/ptr/a...,NaN,P000612,d55dc2ee-29d4-4f37-b6ec-e7bbc25c51b1,116,Not applicable or not available,NaN,this filing was disclosed via scanned pdf. use...,[-1.42796878e-02 -1.07458733e-01 2.85561293e-...,2020-11-04
7957,7957,Sen. David Perdue,david perdue,2020-04-16,05/08/2020,DOW,Dow Inc.,Sale (Full),"$1,001 - $15,000",NaN,...,https://efdsearch.senate.gov/search/view/ptr/b...,1000.0,P000612,56f4ac7e-0f49-4a93-886b-92ac1aa63185,116,Not applicable or not available,NaN,dow inc. [SEP] dow [SEP] not applicable or not...,[-3.78000401e-02 3.04723065e-02 -6.46337271e-...,2020-04-16
7958,7958,Sen. David Perdue,david perdue,2020-04-16,05/08/2020,EEM,"iShares MSCI Emerging Markets ETF <div class=""...",Sale (Full),"$1,001 - $15,000",NaN,...,https://efdsearch.senate.gov/search/view/ptr/b...,1000.0,P000612,c115a643-44de-4f5f-b405-ae08a38c8eeb,116,Not applicable or not available,NaN,"ishares msci emerging markets etf <div class=""...",[ 3.49410325e-02 -8.48345682e-02 3.20679456e-...,2020-04-16
7959,7959,Sen. David Perdue,david perdue,2020-04-16,05/08/2020,IDV,iShares International Select Dividend ETF <div...,Sale (Full),"$1,001 - $15,000",NaN,...,https://efdsearch.senate.gov/search/view/ptr/b...,1000.0,P000612,c6721264-2710-44a8-8e4e-02b311b8c04b,116,Not applicable or not available,NaN,ishares international select dividend etf <div...,[ 1.38572395e-01 -7.44563788e-02 2.55563825e-...,2020-04-16
7960,7960,Sen. David Perdue,david perdue,2020-04-14,05/08/2020,DD,"DuPont de Nemours, Inc.",Sale (Full),"$50,001 - $100,000",NaN,...,https://efdsearch.senate.gov/search/view/ptr/b...,50000.0,P000612,e98153dc-4ead-4720-9ae8-e706c285c644,116,Not applicable or not available,NaN,"dupont de nemours, inc. [SEP] dd [SEP] not app...",[-5.74763166e-03 3.64099331e-02 -9.31705609e-...,2020-04-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13361,13361,Sen. David Perdue,david perdue,2015-05-13,06/04/2015,DISCA,"Discovery Communications, Inc. (NASDAQ)",Purchase,"$1,001 - $15,000",Television Services,...,https://efdsearch.senate.gov/search/view/ptr/1...,1000.0,P000612,4359e9f1-bd19-4815-b083-cac1e01bdd7c,116,Companies providing television services and re...,The Consumer Services sector includes business...,"discovery communications, inc. (nasdaq) [SEP] ...",[ 1.74970970e-01 3.68919939e-01 4.37433366e-...,2015-05-13
13362,13362,Sen. David Perdue,david perdue,2015-05-14,06/04/2015,DISCA,"Discovery Communications, Inc. (NASDAQ)",Purchase,"$1,001 - $15,000",Television Services,...,https://efdsearch.senate.gov/search/view/ptr/1...,1000.0,P000612,7b1372e4-7157-4a48-a93e-9120c1343718,116,Companies providing television services and re...,The Consumer Services sector includes business...,"discovery communications, inc. (nasdaq) [SEP] ...",[ 1.74970970e-01 3.68919939e-01 4.37433366e-...,2015-05-14
13363,13363,Sen. David Perdue,david perdue,2015-05-14,06/04/2015,TAP,Molson Coors Brewing Company (NYSE),Sale (Full),"$1,001 - $15,000",Beverages (Production/Distribution),...,https://efdsearch.senate.gov/search/view/ptr/1...,1000.0,P000612,ec28f9af-2e47-4611-b9fa-6be2635390ba,116,Companies engaged in the production and distri...,The Consumer Non-Durables sector encompasses c...,molson coors brewing company (nyse) [SEP] tap ...,[-2.90675219e-02 3.76925081e-01 -3.04134432e-...,2015-05-14
13364,13364,Sen. David Perdue,david perdue,2015-05-15,06/04/2015,AXLL,Axiall Corporation (NYSE),Purchase,"$1,001 - $15,000",NaN,...,https://efdsearch.senate.gov/search/view/ptr/1...,1000.0,P000612,88bfbbf4-fd08-46f5-8724-13796be1bb7e

In [126]:
transaction_of_interest = 'b3d55c8a-8136-46b9-b9c8-72cbbd550b82'
transaction_date = '2020-02-27'
transaction_embedding = transactions[transactions['uuid'] == transaction_of_interest]['embedding'].values[0]
transaction_embedding = ast.literal_eval(clean_array_string(transaction_embedding))
len(transaction_embedding)

768

In [127]:
politician_id = transactions[transactions['display_name'] == politician_selection]['member_id'].values[0]
politician_congress = transactions[transactions['display_name'] == politician_selection]['congress'].values[0]

In [128]:
"""filter committee and subcommittee assignments by politician and congress"""

politician_committees = committee_assignments[(committee_assignments['member_id'] == politician_id) & (committee_assignments['congress'] == politician_congress)]
politician_committees['clean_embeddings'] = politician_committees['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x)))

politician_subcommittees = subcommittee_assignments[(subcommittee_assignments['member_id'] == politician_id) & (subcommittee_assignments['congress'] == politician_congress)]
politician_subcommittees['clean_embeddings'] = politician_subcommittees['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x)))

In [129]:

"""Get cosine similarity for each committee"""
committee_hearings_emb_matrix = np.vstack(politician_committees['clean_embeddings'].values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)
politician_committees['similarity'] = cosine_similarity(committee_hearings_emb_matrix,transaction_emb_matrix)
politician_committees

,Unnamed: 0.1,Unnamed: 0,member_id,member_name,member_begin_date,congress,chamber,committee_id,committee_name,committee_description,string_object,uuid,embedding,clean_embeddings,similarity
7,7,7,P000612,David Perdue,2019-01-09,116,senate,SSAF,"Committee on Agriculture, Nutrition, and Forestry","The Committee on Agriculture, Nutrition, and F...","david perdue[SEP] committee on agriculture, nu...",b3547b8a-ab95-46a3-a07d-b9537de11bd3,[-2.38100842e-01 2.23639682e-01 -1.72280565e-...,"[-0.238100842, 0.223639682, -0.172280565, -0.0...",0.636740
42,42,42,P000612,David Perdue,2019-01-09,116,senate,SSAS,Committee on Armed Services,The House Committee on Armed Services and its ...,david perdue[SEP] committee on armed services[...,4e0b8595-2e8f-4e8b-87e2-a593745627e6,[-2.59135038e-01 6.47039339e-02 -2.49970615e-...,"[-0.259135038, 0.0647039339, -0.249970615, -0....",0.656170
55,55,55,P000612,David Perdue,2019-01-09,116,senate,SSBK,"Committee on Banking, Housing, and Urban Affairs","The Committee on Banking, Housing, and Urban A...","david perdue[SEP] committee on banking, housin...",e4c39681-07de-4962-84f3-30dae1b5c84f,[-2.11296827e-01 1.85890391e-01 3.66631523e-...,"[-0.211296827, 0.185890391, 0.0366631523, -0.0...",0.678430
188,188,188,P000612,David Perdue,2019-01-09,116,senate,SSBU,Committee on the Budget,The Committee on the Budget is a standing comm...,david perdue[SEP] committee on the budget[SEP]...,dbdd9f84-6c0f-4c8b-8596-6a64f0087c0e,[-3.90885115e-01 9.82047096e-02 -1.15585916e-...,"[-0.390885115, 0.0982047096, -0.115585916, 0.0...",0.630425
238,238,238,P000612,David Perdue,NaN,116,senate,SCNC,United States Caucus on International Narcotic...,The United States Caucus on International Narc...,david perdue[SEP] united states caucus on inte...,c9625c11-c4d8-46e7-b43e-8cac026b80a4,[-3.59756529e-01 -9.09578335e-03 -1.59447998e-...,"[-0.359756529, -0.00909578335, -0.159447998, -...",0.652716


In [130]:
"""Get cosine similarity for each subcommittee"""
subcommittee_hearings_emb_matrix = np.vstack(politician_subcommittees['clean_embeddings'].values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)
politician_subcommittees['similarity'] = cosine_similarity(subcommittee_hearings_emb_matrix,transaction_emb_matrix)
politician_subcommittees

,Unnamed: 0.1,Unnamed: 0,member_id,member_name,member_begin_date,congress,chamber,committee_id,subcommittee_id,subcommittee_name,parent_committee_id,subcommittee_description,string_object,uuid,embedding,clean_embeddings,similarity
9,9,9,P000612,David Perdue,NaN,116,senate,SSAF,SSAF13,"Commodities, Risk Management, and Trade",SSAF,"The Commodities, Risk Management, and Trade su...","David Perdue [SEP] nan [SEP] The Commodities,...",03bc8ddd-43ae-4fce-a7e1-004f9aee5598,[-2.15670258e-01 -3.81724648e-02 -5.88180199e-...,"[-0.215670258, -0.0381724648, -0.0588180199, 0...",0.707618
13,13,13,P000612,David Perdue,NaN,116,senate,SSAF,SSAF14,"Conservation, Climate, Forestry, and Natural R...",SSAF,"The Conservation, Climate, Forestry, and Natur...",David Perdue [SEP] nan [SEP] The Conservation...,0fe6ffab-b88a-4cc9-aeea-cacded8a45b8,[-3.01194936e-01 1.93707943e-01 -4.04974073e-...,"[-0.301194936, 0.193707943, -0.0404974073, 0.0...",0.642790
23,23,23,P000612,David Perdue,NaN,116,senate,SSAF,SSAF17,"Livestock, Dairy, Poultry, Local Food Systems,...",SSAF,"The Livestock, Dairy, Poultry, Local Food Syst...","David Perdue [SEP] nan [SEP] The Livestock, D...",957b0270-aca2-43fe-b9f1-8f54906958e4,[-1.13164999e-01 1.39037564e-01 -9.61981155e-...,"[-0.113164999, 0.139037564, -0.00961981155, -0...",0.704416
173,173,173,P000612,David Perdue,NaN,116,senate,SSAS,SSAS13,Seapower,SSAS,The Seapower subcommittee is a part of the Hou...,David Perdue [SEP] nan [SEP] The Seapower sub...,b6a0b704-9913-495f-9f1d-c0877a9e96af,[-2.64365107e-01 -1.04906186e-01 -1.80850551e-...,"[-0.264365107, -0.104906186, -0.180850551, -0....",0.642809
197,197,197,P000612,David Perdue,NaN,116,senate,SSAS,SSAS15,Readiness and Management Support,SSAS,The Readiness and Management Support subcommit...,David Perdue [SEP] nan [SEP] The Readiness an...,41728ba4-6b3b-4223-a250-74647c3b7d9d,[-3.82136732e-01 6.76125064e-02 -6.36343062e-...,"[-0.382136732, 0.0676125064, -0.0636343062, -0...",0.665026
204,204,204,P000612,David Perdue,NaN,116,senate,SSAS,SSAS21,Cybersecurity,SSAS,The Cybersecurity subcommittee is a subcommitt...,David Perdue [SEP] nan [SEP] The Cybersecurit...,a879ef71-2d13-4a9b-ab1f-a2348b6f7c6c,[-3.97993237e-01 -3.82100185e-03 1.09654153e-...,"[-0.397993237, -0.00382100185, 0.0109654153, 0...",0.651212
216,216,216,P000612,David Perdue,2019-01-18,116,senate,SSBK,SSBK04,"Securities, Insurance, and Investment",SSBK,"The Securities, Insurance, and Investment Subc...",David Perdue [SEP] 2019-01-18 [SEP] The Secur...,4f98cd92-bc17-4fb2-b589-2f58f1c67a79,[-2.65492767e-01 4.69140336e-02 5.67980781e-...,"[-0.265492767, 0.0469140336, 0.0567980781, -0....",0.696066
221,221,221,P000612,David Perdue,2019-01-18,116,senate,SSBK,SSBK09,"Housing, Transportation, and Community Develop...",SSBK,"The Housing, Transportation, and Community Dev...",David Perdue [SEP] 2019-01-18 [SEP] The Housi...,edb52215-5194-499d-a5e1-bd854590e743,[-3.82826954e-01 6.46189004e-02 6.33963719e-...,"[-0.382826954, 0.0646189004, 0.0633963719, -0....",0.644160
229,229,229,P000612,David Perdue,2019-01-18,116,senate,SSBK,SSBK12,Economic Policy,SSBK,The Economic Policy Subcommittee is a subcommi...,David Perdue [SEP] 2019-01-18 [SEP] The Econo...,5a362ee9-f852-4932-b3f2-1ed0275b283c,[-3.67562920e-01 4.93770652e-02 9.60456058e-...,"[-0.36756292, 0.0493770652, 0.0960456058, -0.0...",0.680091


In [131]:
bills = pd.read_csv('final_data/bills_final.csv')
filtered_bills = bills[bills['member_id'] == politician_id]

"""Get cosine similarity for each bill"""
bill_emb_matrix = np.vstack(filtered_bills['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x))).values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)
filtered_bills['similarity'] = cosine_similarity(bill_emb_matrix,transaction_emb_matrix)


In [138]:
politician_committee_assignments = politician_committees['committee_id'].values
committee_hearings = pd.read_csv('final_data/committee_hearings_final.csv')
filtered_hearings = committee_hearings[(committee_hearings['hearing_committee_id'].isin(politician_committee_assignments))& (committee_hearings['hearing_congress'] == politician_congress)]

"""Get cosine similarity for each hearing"""
hearing_emb_matrix = np.vstack(filtered_hearings['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x))).values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)

filtered_hearings['similarity'] = cosine_similarity(hearing_emb_matrix,transaction_emb_matrix)
filtered_hearings

,Unnamed: 0.1,Unnamed: 0,hearing_congress,hearing_chamber,hearing_committee_id,hearing_date,hearing_description,string_object,uuid,embedding,similarity
1239,1239,1239,116.0,senate,SSAF,2020-12-02,Hearings to examine agricultural research and ...,116.0 [SEP] senate [SEP] hearings to examine a...,54a43c57-324c-4111-8ed5-7ef5f40a4d1b,[-5.48893735e-02 -3.98558259e-01 -2.78647423e-...,0.584375
1240,1240,1240,116.0,senate,SSAF,2020-06-24,Business meeting to consider original legislat...,116.0 [SEP] senate [SEP] business meeting to c...,7171f21a-ac13-465f-8c89-867028cc4f75,[ 2.96993684e-02 -1.83329836e-01 3.93226594e-...,0.653401
1241,1241,1241,116.0,senate,SSAF,2020-03-12,Hearings to examine agriculture innovation and...,116.0 [SEP] senate [SEP] hearings to examine a...,82bbd4e6-cb42-4dcb-bab1-8570ce973b34,[-6.20919950e-02 -4.59316432e-01 -1.95373014e-...,0.583406
1242,1242,1242,116.0,senate,SSAF,2019-11-07,Hearings to examine implementation of the 2018...,116.0 [SEP] senate [SEP] hearings to examine i...,4e5611f5-90ad-493f-954e-fb21a6a8fc19,[-3.64600383e-02 -4.97940034e-01 -2.03136593e-...,0.606229
1243,1243,1243,116.0,senate,SSAF,2019-10-17,Hearings to examine implementing the 2018 Farm...,116.0 [SEP] senate [SEP] hearings to examine i...,d3acfa28-42c5-4ddf-9650-d5ddc317db86,[-9.29759964e-02 -6.25922203e-01 -2.09133878e-...,0.531772
...,...,...,...,...,...,...,...,...,...,...,...
1916,1916,1916,116.0,senate,SSBU,2019-03-13,Hearings to examine the President&#39;s propos...,116.0 [SEP] senate [SEP] hearings to examine t...,b8e90c21-7219-47e5-9ce7-e8b58e08e70a,[-2.62802899e-01 -3.69865924e-01 3.49259190e-...,0.572939
1917,1917,1917,116.0,senate,SSBU,2019-02-27,"Hearings to examine the Budget Control Act, fo...",116.0 [SEP] senate [SEP] hearings to examine t...,f0eb4565-ee22-4fec-8501-8f7fa85fb51b,[-9.51932818e-02 -4.92828667e-01 -2.49492362e-...,0.585390
1918,1918,1918,116.0,senate,SSBU,2019-01-29,Hearings to examine the Congressional Budget O...,116.0 [SEP] senate [SEP] hearings to examine t...,67a0fcb0-e2f3-4e88-98a3-c5b0e2a3ba6d,[-3.04276794e-01 -3.01852316e-01 8.20049644e-...,0.592752
2101,2101,2101,116.0,senate,SCNC,2019-10-23,Hearings to examine marijuana and America&#39;...,116.0 [SEP] senate [SEP] hearings to examine m...,7c5ac7dc-dcc0-4ed2-8033-a25bb6baa5c1,[ 6.34304732e-02 -2.92708993e-01 -6.58062622e-...,0.631462


In [140]:
travel = pd.read_csv('final_data/travel_final.csv')
filtered_travel = travel[(travel['member_id'] == politician_id) & (travel['congress'] == politician_congress)]

"""Get cosine similarity for each travel"""
travel_emb_matrix = np.vstack(filtered_travel['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x))).values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)

filtered_travel['similarity'] = cosine_similarity(travel_emb_matrix,transaction_emb_matrix)

filtered_travel

ValueError: need at least one array to concatenate

In [141]:
related_bills = pd.read_csv('final_data/related_bills_final.csv')
filtered_related_bills = related_bills[(related_bills['member_id'] == politician_id) & (related_bills['bill_congress'] == politician_congress)]

"""Get cosine similarity for each related bill"""
related_bill_emb_matrix = np.vstack(filtered_related_bills['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x))).values)
transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)

filtered_related_bills['similarity'] = cosine_similarity(related_bill_emb_matrix,transaction_emb_matrix)

filtered_related_bills

,Unnamed: 0.1,Unnamed: 0,related_bill_id,related_bill_title,related_bill_introduction_date,bill_slug,bill_congress,member_id,string_object,uuid,embedding,similarity
6153,6153,6153,sres726-116,A resolution honoring Rosalynn Smith Carter's ...,2020-09-29,sres726,116,P000612,a resolution honoring rosalynn smith carter's ...,3583cbe4-afaf-4e74-b9b5-3917a64095d8,[-3.04253250e-01 -3.56172532e-01 -3.58239026e-...,0.579690
6154,6154,6154,s4024-116,A bill to establish in the Cybersecurity and I...,2020-06-22,s4024,116,P000612,a bill to establish in the cybersecurity and i...,3e534dec-0331-427c-a927-3493df9b771d,[-2.57931411e-01 -3.38810474e-01 -1.02268927e-...,0.671624
6155,6155,6155,s3599-116,A bill to enhance our Nation's nurse and physi...,2020-05-05,s3599,116,P000612,a bill to enhance our nation's nurse and physi...,6a74f573-9339-485b-b027-a7e7e055da8f,[-1.43147171e-01 -3.37306321e-01 6.43994659e-...,0.669626
6156,6156,6156,s3098-116,A bill to redesignate the Jimmy Carter Nationa...,2019-12-18,s3098,116,P000612,a bill to redesignate the jimmy carter nationa...,b9a68da0-21da-4929-92fb-5d8862adb20c,[-2.35384271e-01 -3.36135954e-01 3.92684946e-...,0.615192
6157,6157,6157,s2025-116,A bill to amend the Motor Carrier Safety Impro...,2019-06-27,s2025,116,P000612,a bill to amend the motor carrier safety impro...,cce699f7-9c2c-43b7-a977-d5350bbfdab1,[ 2.98766624e-02 -3.33784074e-01 -2.89239317e-...,0.660259
6158,6158,6158,s1898-116,"A bill to amend title 5, United States Code, t...",2019-06-19,s1898,116,P000612,"a bill to amend title 5, united states code, t...",73754595-8a0d-4af6-b173-c4c6e7cc69a8,[ 1.75114930e-01 -1.67265609e-01 -3.68312299e-...,0.643079
6159,6159,6159,s1807-116,A bill to improve the funding process.,2019-06-12,s1807,116,P000612,a bill to improve the funding process. [SEP] 1...,418ce59c-e4eb-4f52-835f-f2ed6f802b84,[-3.03273082e-01 -6.66067004e-01 -1.34174585e-...,0.451968
6160,6160,6160,s1677-116,A bill to amend the Internal Revenue Code of 1...,2019-05-23,s1677,116,P000612,a bill to amend the internal revenue code of 1...,5945f805-9efd-492d-a5d8-6859fdb0f314,[-2.16982737e-02 -3.11814666e-01 -1.83130756e-...,0.634503
6161,6161,6161,sres182-116,A resolution recognizing the duty of the Senat...,2019-05-01,sres182,116,P000612,a resolution recognizing the duty of the senat...,2cf1041b-8e04-4521-b88e-9306712a1527,[-1.31731465e-01 -2.14626268e-01 -1.72572583e-...,0.611991
6162,6162,6162,s1078-116,"A bill to amend title 44, United States Code, ...",2019-04-09,s1078,116,P000612,"a bill to amend title 44, united states code, ...",5e7cf625-37ae-49a5-8e39-585de5aeecc1,[ 9.08652693e-02 -1.43209770e-01 -3.03162873e-...,0.629096


In [142]:
member_statements = pd.read_csv('final_data/member_statements_final.csv')

filtered_statements = member_statements[(member_statements['member_id'] == politician_id) & (member_statements['congress'] == politician_congress)]


"""Get cosine similarity for each statement"""
statement_emb_matrix = np.vstack(filtered_statements['embedding'].apply(lambda x: ast.literal_eval(clean_array_string(x))).values)

transaction_emb_matrix = np.array(transaction_embedding).reshape(1,-1)

filtered_statements['similarity'] = cosine_similarity(statement_emb_matrix,transaction_emb_matrix)

filtered_statements

,Unnamed: 0.1,Unnamed: 0,url,date,title,type,member_id,congress,string_object,uuid,embedding,similarity
56493,56493,56493,https://www.perdue.senate.gov/news/press-relea...,2020-11-17,Senator David Perdue On Third Annual Departmen...,Press Release,P000612,116,senator david perdue on third annual departmen...,15e26673-9f1a-463f-94b8-461ce5d4730d,[-1.50185436e-01 -2.51212567e-01 -1.61012620e-...,0.599659
56494,56494,56494,https://www.perdue.senate.gov/news/press-relea...,2020-11-12,"Senators Perdue, Loeffler Introduce SAFE Votes...",Press Release,P000612,116,"senators perdue, loeffler introduce safe votes...",c492ff4e-6f80-4c55-be3c-bc8d9e8ba149,[ 1.94357052e-01 -2.36676365e-01 -2.21671283e-...,0.571226
56495,56495,56495,https://www.perdue.senate.gov/news/press-relea...,2020-11-11,Senator David Perdue Honors Georgia’s Veterans,Press Release,P000612,116,senator david perdue honors georgia’s veterans...,39e75973-5e2c-493d-bb24-013dc138ed02,[-9.07850713e-02 -1.77687004e-01 -1.26303732e-...,0.555928
56496,56496,56496,https://www.perdue.senate.gov/news/press-relea...,2020-11-10,Senator David Perdue Takes Action To Ensure Fa...,Press Release,P000612,116,senator david perdue takes action to ensure fa...,8587e695-debc-4ec0-9348-f67704901dc4,[-1.04975194e-01 -3.53616506e-01 -3.18290442e-...,0.592220
56497,56497,56497,https://www.perdue.senate.gov/news/press-relea...,2020-11-10,Senator David Perdue On Protecting Senate Majo...,Press Release,P000612,116,senator david perdue on protecting senate majo...,aaab436d-dfd9-43de-805e-688eb7581e00,[ 1.66590884e-02 -4.16283160e-01 -2.47606277e-...,0.535806
...,...,...,...,...,...,...,...,...,...,...,...,...
56888,56888,56888,https://www.perdue.senate.gov/news/press-relea...,2020-10-08,Senator David Perdue Fights For Georgia Veterans,Press Release,P000612,116,senator david perdue fights for georgia vetera...,cffebc16-5082-4367-a233-9ed7f64db1eb,[ 1.27461571e-02 -3.80418897e-01 -8.70815367e-...,0.529003
56889,56889,56889,https://www.perdue.senate.gov/news/press-relea...,2020-10-06,Roswell-Based Foster Care Support Foundation R...,Press Release,P000612,116,roswell-based foster care support foundation r...,2014325b-a4f7-4b6b-be5a-a5505416dfe3,[-1.02277443e-01 -2.93947279e-01 1.15610227e-...,0.627817
56890,56890,56890,https://www.perdue.senate.gov/news/press-relea...,2020-09-30,Senator David Perdue Votes To Protect American...,Press Release,P000612,116,senator david perdue votes to protect american...,d5169518-3b64-4c90-b6a7-4c4ab232140a,[-4.72655334e-03 -2.69629270e-01 -2.60442555e-...,0.584382
56891,56891,56891,https://www.perdue.senate.gov/news/press-relea...,2020-09-30,Senator David Perdue On Funding The Government,Press Release,P000612,116,senator david perdue on funding the government...,fe2b993f-361b-48c5-b5a0-800fd4de49de,[ 4.49101031e-02 -2.38537133e-01 -2.87175536e-...,0.550474
